In [5]:
import numpy as np
import pandas as pd
from tqdm import tqdm
%matplotlib inline
import matplotlib.pyplot as plt

Simulated Experiment:
- Generate 1000 risk scores ~ Uniform(0,1)
- Simulate true outcome = Bernoulli(risk) 
- Threshold prediction = 1 if risk >= 0.5, 0 else
- Random prediction = Bernoulli(risk) 

If multiple decision-makers (models) estimate the same risk, does homogenization improve with random predictions? 
- Each model flips its own Bernoulli coin instead of using the same threshold
- Overall experiment conducted 1000 times  

Partially Random Classifier:
- Use random prediction if risk between some upper and lower bound (e.g. 0.4 to 0.6)
- Otherwise use threshold prediction

# Experiment

In [6]:
class Tracker:
    def __init__(self, random_distance, num_models, size=1000):
        self.random_distance = random_distance
        self.num_models = num_models
        
        self.accuracy = []
        self.acceptance = []
        
        self.systemic_success = (np.ones(size)==1)
        self.systemic_failure = (np.ones(size)==1)
        
        self.failure_rate = 1
    
    def get_predictions(self, risk, random_pred, threshold_pred):
        pred = []
        for i in range(len(risk)):
            if (risk[i]>0.5-self.random_distance) and (risk[i]<0.5+self.random_distance):
                pred.append(random_pred[i])
            else:
                pred.append(threshold_pred[i])
        return np.array(pred)
    
    def update_metrics(self, risk, random_pred, threshold_pred, outcomes):
        pred = self.get_predictions(risk, random_pred, threshold_pred)
        
        self.accuracy.append(np.sum(pred==outcomes)/len(pred))
        self.acceptance.append(np.sum(pred)/len(pred))

        self.failure_rate *= np.sum(pred==0)/len(pred)
        
        self.systemic_success *= (pred==1)
        self.systemic_failure *= (pred==0)

    def final_metrics(self):
        r = {}
        r["random_distance"] = self.random_distance
        r["num_models"] = self.num_models
        
        r["accuracy"] = np.mean(self.accuracy)
        r["acceptance"] = np.mean(self.acceptance)
        
        r["systemic_success"] = np.sum(self.systemic_success)/len(self.systemic_success)
        r["systemic_failure"] = np.sum(self.systemic_failure)/len(self.systemic_failure)
        r["multiplicity"] = 1-r["systemic_success"]-r["systemic_failure"]
        
        r["failure_rate"] = self.failure_rate
        r["homogenization_expected_failure"] = r["systemic_failure"]/self.failure_rate
        r["homogenization_avg_failure"] = r["systemic_failure"]/(1-r["acceptance"])
        
        return r

In [7]:
def experiment(num_models=2):
    results = []
    
    for i in range(1000):
        risk = np.random.uniform(0,1,1000)
        
        outcomes = []
        for r in risk:
            outcomes.append(np.random.binomial(1, r))
        
        threshold = Tracker(0, num_models)
        random_10 = Tracker(0.1, num_models)
        random_20 = Tracker(0.2, num_models)
        random_30 = Tracker(0.3, num_models)
        random_40 = Tracker(0.4, num_models)
        random = Tracker(0.5, num_models)
        
        for k in range(num_models):
            random_pred = []
            threshold_pred = []

            for r in risk:
                random_pred.append(np.random.binomial(1, r))
                if r>=0.5:
                    threshold_pred.append(1)
                else:
                    threshold_pred.append(0)
            
            threshold.update_metrics(risk, random_pred, threshold_pred, outcomes)
            random_10.update_metrics(risk, random_pred, threshold_pred, outcomes)
            random_20.update_metrics(risk, random_pred, threshold_pred, outcomes)
            random_30.update_metrics(risk, random_pred, threshold_pred, outcomes)
            random_40.update_metrics(risk, random_pred, threshold_pred, outcomes)
            random.update_metrics(risk, random_pred, threshold_pred, outcomes)
        
        results.append(threshold.final_metrics())
        results.append(random_10.final_metrics())
        results.append(random_20.final_metrics())
        results.append(random_30.final_metrics())
        results.append(random_40.final_metrics())
        results.append(random.final_metrics())
        
    return results

In [8]:
results = []
for k in tqdm(range(1, 11)):
    results += experiment(k)
df = pd.DataFrame(results)

In [9]:
df = df.groupby(["random_distance", "num_models"]).mean().reset_index(drop=False)

# Analysis

In [10]:
def plot(df, title, metric, ylog=False):
    plt.rcParams["figure.figsize"] = [7,3]
    fig, ax = plt.subplots()

    plt.plot(df.loc[df["random_distance"]==0, "num_models"], df.loc[df["random_distance"]==0, metric], '*-', color='#00060e', label="Threshold")
    plt.plot(df.loc[df["random_distance"]==0.1, "num_models"], df.loc[df["random_distance"]==0.1, metric], '*-', color='#001e49', label="Random(0.4,0.6)")
    plt.plot(df.loc[df["random_distance"]==0.2, "num_models"], df.loc[df["random_distance"]==0.2, metric], '*-', color='#002f70', label="Random(0.3,0.7)")
    plt.plot(df.loc[df["random_distance"]==0.3, "num_models"], df.loc[df["random_distance"]==0.3, metric], '*-', color='#003f97', label="Random(0.2,0.8)")
    plt.plot(df.loc[df["random_distance"]==0.4, "num_models"], df.loc[df["random_distance"]==0.4, metric], '*-', color='#004fbf', label="Random(0.1,0.9)")
    plt.plot(df.loc[df["random_distance"]==0.5, "num_models"], df.loc[df["random_distance"]==0.5, metric], '*-', color='#0068f9', label="All Random")
  
    #plt.ylim(0,1)
    if ylog:
        plt.yscale('log')   

    plt.xlabel('Number of models')
    plt.ylabel(metric)
    plt.title(title)
    plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
    fig.tight_layout()
    plt.show()

In [11]:
plot(df, "Accuracy", "accuracy")

In [12]:
plot(df, "Systemic Failure", "systemic_failure")

In [13]:
plot(df, "Systemic Success", "systemic_success")

In [14]:
plot(df, "Multiplicity", "multiplicity")

In [15]:
plot(df, "Acceptance Rate", "acceptance")

In [16]:
plot(df, "Homogenization (Systemic Failure / Expected Failure)", "homogenization_expected_failure", True)

In [17]:
plot(df, "Homogenization (Systemic Failure / Expected Failure)", "homogenization_expected_failure", False)

In [18]:
plot(df, "Expected Failure Rate", "failure_rate", False)

In [19]:
plot(df, "Homogenization (Systemic Failure / Average Failure)", "homogenization_avg_failure")